In [ ]:
%run functions/cytoscapeFunctions
%run functions/edgeFunctions
%run functions/springLayoutJN-Copy1

In [ ]:
from py2cytoscape import cyrest
import networkx as nx
import numpy as np
import pandas as pd
import random
import time
import tempfile
from networkx.generators.random_graphs import barabasi_albert_graph as ba

In [ ]:
dirPath = os.path.dirname(os.path.abspath('template.cys'))

In [ ]:
cytoscape = cyrest.cyclient()

In [ ]:
cytoscape.status() 

In [ ]:
from py2cytoscape.data.cyrest_client import CyRestClient
cy = CyRestClient()

In [ ]:
cy.session.delete()

In [ ]:
import os
cytoscape.session.open(session_file = os.path.abspath("template.cys"))

In [ ]:
# G = nx.fast_gnp_random_graph(15, 0.2)
# G1 = nx.fast_gnp_random_graph(10, 0.3)
# addEdgeAttrib(G, 'weight', 25,30)
# addEdgeAttrib(G, 'color',5,200)
# addEdgeAttrib(G1, 'weight',30,35)
# addEdgeAttrib(G1, 'color',5,200)

In [ ]:
networkFolder = os.path.join(os.path.dirname(os.path.abspath("template.cys")), 'networks')
networkFolder

GMP = os.path.join(networkFolder, 'GMP')
Prog = os.path.join(networkFolder, 'Progenitor')

GMPedge = os.path.join(GMP, 'edge.tsv')
GMPnode = os.path.join(GMP, 'node.tsv')

ProgEdge = os.path.join(Prog, 'edge.tsv')
ProgNode = os.path.join(Prog, 'node.tsv')

gmpEdgeDF = pd.read_csv(GMPedge, delimiter = '\t')
gmpNodeDF = pd.read_csv(GMPnode, delimiter = '\t')
progEdgeDF = pd.read_csv(ProgEdge, delimiter = '\t')
progNodeDF = pd.read_csv(ProgNode, delimiter = '\t')

gmpNetwork = nx.Graph()
gmpOverlap = 0

for i in range (len(gmpEdgeDF)):
    if gmpNetwork.has_edge(gmpEdgeDF['Regulator'][i], gmpEdgeDF['Target'][i]):
        gmpOverlap += 1
    gmpNetwork.add_edge(gmpEdgeDF['Regulator'][i], gmpEdgeDF['Target'][i])
    
progNetwork = nx.Graph()

progOverlap = 0

for i in range (len(progEdgeDF)):
    if progNetwork.has_edge(progEdgeDF['Regulator'][i], progEdgeDF['Target'][i]):
        progOverlap += 1
    progNetwork.add_edge(progEdgeDF['Regulator'][i], progEdgeDF['Target'][i])

print(gmpOverlap, progOverlap)    

gmpNodeIndexDF = gmpNodeDF.set_index('Name')
progNodeIndexDF = progNodeDF.set_index('Name')

nx.set_node_attributes(gmpNetwork, gmpNodeDF.to_dict('index'))
nx.set_node_attributes(gmpNetwork, gmpNodeIndexDF.to_dict('index'))

gmpNetwork1 = gmpNetwork.copy()
# gmpNetwork1.remove_node('IRF7')

nx.set_node_attributes(progNetwork, progNodeDF.to_dict('index'))
nx.set_node_attributes(progNetwork, progNodeIndexDF.to_dict('index'))

In [ ]:
addEdgeAttrib(gmpNetwork, 'weight',0,1)
addEdgeAttrib(gmpNetwork1, 'weight', 0, 1)

In [ ]:
addEdgeAttrib(progNetwork, 'weight', 17,23)

In [ ]:
conformEdges(gmpNetwork, gmpNetwork1)
# conformEdges(gmpNetwork, progNetwork)
networks = fillBetweenEdges(gmpNetwork, gmpNetwork1, 10)
# networks2 = fillBetweenEdges(gmpNetwork, progNetwork, 10)

In [ ]:
deleteZeroWeights(networks)

In [ ]:
importEdge(convertEdge(networks[0]), cytoscape)

In [ ]:
importNode(convertNode(networks[0]), cytoscape)

In [ ]:
deleteLoneListNodes(networks)

In [ ]:
convertNode(networks[0]).index

In [ ]:
layout = fruchterman_reingold_layout_edit(networks[0], seed = 1,iterations = 1000, pretendIterations = 50)
layoutList = list(layout.values())
for x in range(len(layoutList)):
    for y in range (2):
        layoutList[x][y] = (layoutList[x][y]*1250)
layoutPD = pd.DataFrame(layoutList, columns = ['x','y'],index = convertNode(networks[0]).index)
nx.draw(networks[0], pos = layout)

In [ ]:
toCytoscape(convertNode(networks[0]), convertEdge(networks[0]), cytoscape)

In [ ]:
importNode(layoutPD, cytoscape)

In [ ]:
layout1 = fruchterman_reingold_layout_edit(networks[1], pos = layout, seed = 1, iterations = 1000, pretendIterations = 50, stop = 15)
nx.draw(networks[1], pos = layout1, with_labels = False)

In [ ]:
layout2 = fruchterman_reingold_layout_edit(networks[2], pos = layout1, seed = 1, iterations = 1000, pretendIterations = 50, stop = 15)
nx.draw(networks[9], pos = layout2, with_labels = False)

In [ ]:
importNode(layoutPD, cytoscape)

In [ ]:
nx.draw(networks[0], pos = layout)

In [ ]:
convertEdge(networks[0])

In [ ]:
layoutPD

In [ ]:
importNode(layoutPD, cytoscape)

In [ ]:
# for network in networks:
#     nx.set_node_attributes(network, gmpNodeDF.to_dict('index'))
#     nx.set_node_attributes(network, gmpNodeIndexDF.to_dict('index'))
    
gmpNames = convertNode(gmpNetwork).index

In [ ]:
sampLayout = fruchterman_reingold_layout_edit(networks[0], seed = 1, iterations = 1000, pretendIterations = 50, stop = 100)
sampLayoutL = list(sampLayout.values())
for i in range(len(sampLayoutL)):
    for j in range (2):
        sampLayoutL[i][j] = (sampLayoutL[i][j]*1000)
        
sampLayoutPD = pd.DataFrame(sampLayoutL, columns = ['x', 'y'], index = gmpNames)

In [ ]:
sampLayoutPD

In [ ]:
sampLayout1 = fruchterman_reingold_layout_edit(networks[2], seed = 1, iterations = 1000, pretendIterations = 50, stop = 100)
sampLayoutL1 = list(sampLayout1.values())
for i in range(len(sampLayoutL1)):
    for j in range (2):
        sampLayoutL1[i][j] = (sampLayoutL1[i][j]*1000)
        
sampLayoutPD1 = pd.DataFrame(sampLayoutL1, columns = ['x', 'y'], index = gmpNames)

In [ ]:
convertEdge(networks[1])

In [ ]:
sampLayoutPD1

In [ ]:
importEdge(convertEdge(networks[0]), cytoscape)
importNode(convertNode(networks[0]), cytoscape)
importNode(sampLayoutPD, cytoscape)

In [ ]:
cytoscape.network.create_empty(RootNetworkList = ' -- Create new network collection --')
importEdge(convertEdge(networks[1]), cytoscape)
importNode(convertNode(networks[1]), cytoscape)
importNode(sampLayoutPD1, cytoscape)

In [ ]:
convertNode(networks[1])